# 1. Information about the submission

## 1.1 Name and number of the assignment 

Assignment 3: Taxonomy Enrichment

## 1.2 Student name

Kundyz Onlabek

## 1.3 Codalab user ID

inflover

# 2. Technical Report

## 2.1 Methodology 

FastText is a library for learning of word embeddings and text classification created by Facebook's AI Research (FAIR) lab. The model allows one to create an unsupervised learning or supervised learning algorithm for obtaining vector representations for words.

Fasttext baseline:
- Compute embeddings of all synsets in RuWordNet by averaging embeddings of all words from senses belonging to a synset.
- Get embeddings for orphans. For multi-word orphans the embeddings are computed by averaging vectors for all words comprising an orphan.
- For each orphan compute the top k = 10 closest synsets of the same part of speech as the orphan using the cosine similarity measure.
- Extract hypernyms for each of these closest synsets from the previous step. Take the first n = 10 results (as each synset may have several hypernyms).

ELMo ("Embeddings from Language Model") is a word embedding method for representing a sequence of words as a corresponding sequence of vectors. Character-level tokens are taken as the inputs to a bi-directional LSTM which produces word-level embeddings. Like BERT (but unlike the word embeddings produced by "Bag of Words" approaches, and earlier vector approaches such as Word2Vec and GloVe), ELMo embeddings are context-sensitive, producing different representations for words that share the same spelling but have different meanings (homonyms) such as "bank" in "river bank" and "bank balance".

## 2.2 Discussion of results

ElMo didn't work at all, randomly suggesting the corresponding hypernyms. Fasttext showed very good results. 

# 3. Code

## Fasttext baseline

In [1]:
!git clone https://github.com/dialogue-evaluation/taxonomy-enrichment.git taxonomy_enrichment
!cd taxonomy_enrichment/; git checkout 6ee12174163368f38276b6e58e81c21625660e13
!mkdir ruwordnet && unzip taxonomy_enrichment/data/ruwordnet.zip -d ruwordnet

Cloning into 'taxonomy_enrichment'...
remote: Enumerating objects: 204, done.
remote: Total 204 (delta 0), reused 0 (delta 0), pack-reused 204
Receiving objects: 100% (204/204), 19.11 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Note: checking out '6ee12174163368f38276b6e58e81c21625660e13'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 6ee1217 update link to the corpus
Archive:  taxonomy_enrichment/data/ruwordnet.zip
  inflating: ruwordnet/composed_of.xml  
  inflating: ruwordnet/derived_from.xml  
  inflating: ruwordnet/senses.A.xml  
  inflating: ruwordnet/senses.N.xml  
  inflating: ruwordnet/senses

In [2]:
import sys
sys.path.append('taxonomy_enrichment/baselines')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p /content/drive_blue
!google-drive-ocamlfuse /content/drive_blue

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [5]:
!nvidia-smi

Tue Dec 21 23:30:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz
!gunzip cc.ru.300.bin.gz

--2021-12-21 23:30:47--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4496459151 (4.2G) [application/octet-stream]
Saving to: ‘cc.ru.300.bin.gz’

cc.ru.300.bin.gz    100%[===================>]   4.19G  12.1MB/s    in 5m 58s  

2021-12-21 23:36:45 (12.0 MB/s) - ‘cc.ru.300.bin.gz’ saved [4496459151/4496459151]



In [7]:
!pip install fasttext

     |████████████████████████████████| 68 kB 5.3 MB/s 
  Using cached pybind11-2.8.1-py2.py3-none-any.whl (208 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3120201 sha256=601430624d0cb6458b4ebe06e0cb091dc9721386ce63c0b5627052e0f6fe3b7b
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [8]:
from collections import defaultdict

import numpy as np
import fasttext.util
from string import punctuation
from ruwordnet.ruwordnet_reader import RuWordnet


class FasttextVectorizer:
    def __init__(self, model_path):
        self.model = fasttext.load_model(model_path)
        print('Model loaded')

    # -------------------------------------------------------------
    # vectorize ruwordnet
    # -------------------------------------------------------------

    def vectorize_ruwordnet(self, synsets, output_path):
        ids, vectors = self.__get_ruwordnet_vectors(synsets)
        self.save_as_w2v(ids, vectors, output_path)

    def __get_ruwordnet_vectors(self, synsets):
        ids = []
        vectors = np.zeros((len(synsets), self.model.get_dimension()))
        for i, (_id, texts) in enumerate(synsets.items()):
            ids.append(_id)
            vectors[i, :] = self.__get_avg_vector(texts)
        return ids, vectors

    def __get_avg_vector(self, texts):
        sum_vector = np.zeros(self.model.get_dimension())
        for text in texts:
            words = [i.strip(punctuation) for i in text.split()]
            sum_vector += np.sum(self.__get_data_vectors(words), axis=0)/len(words)
        return sum_vector/len(texts)

    # -------------------------------------------------------------
    # vectorize data
    # -------------------------------------------------------------

    def vectorize_data(self, data, output_path):
        data_vectors = self.__get_data_vectors(data)
        self.save_as_w2v(data, data_vectors, output_path)

    def __get_data_vectors(self, data):
        vectors = np.zeros((len(data), self.model.get_dimension()))
        for i, word in enumerate(data):  # TODO: how to do it more effective or one-line
            vectors[i, :] = self.model[word]
        return vectors

    # -------------------------------------------------------------
    # save
    # -------------------------------------------------------------

    @staticmethod
    def save_as_w2v(words: list, vectors: np.array, output_path: str):
        assert len(words) == len(vectors)
        with open(output_path, 'w', encoding='utf-8') as w:
            w.write(f"{vectors.shape[0]} {vectors.shape[1]}\n")
            for word, vector in zip(words, vectors):
                vector_line = " ".join(map(str, vector))
                w.write(f"{word.upper()} {vector_line}\n")


def process_data(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        dataset = f.read().lower().split("\n")[:-1]
    ft_vec.vectorize_data(dataset, output_file)

In [9]:
from bs4 import BeautifulSoup
import os
import codecs

from ruwordnet.database import DatabaseRuWordnet

from taxonomy_enrichment.baselines.ruwordnet.ruwordnet_reader import *


class RuWordnet(DatabaseRuWordnet):
    def __init__(self, db_path, ruwordnet_path, with_lemmas=False):
        super(RuWordnet, self).__init__(db_path)
        self.with_lemmas = with_lemmas
        self.__initialize_db(ruwordnet_path)

    def __initialize_db(self, path):
        if self.is_empty():
            print("Inserting data to database")
            synset_files, relation_files, senses_files = get_wordnet_files_from_path(path)

            synsets = [synset for file in synset_files for synset in parse_synsets(file)]
            relations = [relation for file in relation_files for relation in parse_relations(file)]

            if self.with_lemmas:
                senses = [sense for file in synset_files for sense in parse_senses_lemmas(file)]
            else:
                senses = [sense for file in senses_files for sense in parse_senses(file)]

            self.insert_synsets(synsets)
            self.insert_relations(relations)
            self.insert_senses(senses)

In [12]:
ft_vec = FasttextVectorizer("cc.ru.300.bin")

Model loaded


In [13]:
ruwordnet = RuWordnet(db_path="ruwordnet.db", ruwordnet_path="ruwordnet")

In [14]:
noun_synsets = defaultdict(list)
verb_synsets = defaultdict(list)
for sense_id, synset_id, text in ruwordnet.get_all_senses():
    if synset_id.endswith("N"):
        noun_synsets[synset_id].append(text.lower())
    elif synset_id.endswith("V"):
        verb_synsets[synset_id].append(text.lower())

In [15]:
ft_vec.vectorize_ruwordnet(noun_synsets, "ruwordnet_nouns.txt")
ft_vec.vectorize_ruwordnet(verb_synsets, "ruwordnet_verbs.txt")

In [16]:
process_data("taxonomy_enrichment/data/public_test/verbs_public.tsv", "verbs_public.txt")
process_data("taxonomy_enrichment/data/public_test/nouns_public.tsv", "nouns_public.txt")
process_data("taxonomy_enrichment/data/private_test/verbs_private.tsv", "verbs_private.txt")
process_data("taxonomy_enrichment/data/private_test/nouns_private.tsv", "nouns_private.txt")

In [17]:
params = {
  "ruwordnet_vectors_path": "ruwordnet_nouns.txt",
  "data_vectors_path": "nouns_public.txt",
  "test_path": "taxonomy_enrichment/data/public_test/nouns_public.tsv",
  "output_path": "predicted_public_nouns_fasttext.tsv",
  "db_path": "ruwordnet.db",
  "ruwordnet_path": None
}

from taxonomy_enrichment.baselines.main import *

with open(params['test_path'], 'r', encoding='utf-8') as f:
    test_data = f.read().split("\n")[:-1]
  
baseline = SecondOrderModel(params)
results = baseline.predict_hypernyms(list(test_data))
save_to_file(results, params['output_path'], baseline.ruwordnet)

In [18]:
params = {
  "ruwordnet_vectors_path": "ruwordnet_verbs.txt",
  "data_vectors_path": "verbs_public.txt",
  "test_path": "taxonomy_enrichment/data/public_test/verbs_public.tsv",
  "output_path": "predicted_public_verbs_fasttext.tsv",
  "db_path": "ruwordnet.db",
  "ruwordnet_path": None
}

from taxonomy_enrichment.baselines.main import *

with open(params['test_path'], 'r', encoding='utf-8') as f:
    test_data = f.read().split("\n")[:-1]
  
baseline = SecondOrderModel(params)
results = baseline.predict_hypernyms(list(test_data))
save_to_file(results, params['output_path'], baseline.ruwordnet)

In [19]:
params = {
  "ruwordnet_vectors_path": "ruwordnet_nouns.txt",
  "data_vectors_path": "nouns_private.txt",
  "test_path": "taxonomy_enrichment/data/private_test/nouns_private.tsv",
  "output_path": "predicted_private_nouns_fasttext.tsv",
  "db_path": "ruwordnet.db",
  "ruwordnet_path": None
}

from taxonomy_enrichment.baselines.main import *

with open(params['test_path'], 'r', encoding='utf-8') as f:
    test_data = f.read().split("\n")[:-1]
  
baseline = SecondOrderModel(params)
results = baseline.predict_hypernyms(list(test_data))
save_to_file(results, params['output_path'], baseline.ruwordnet)

In [20]:
params = {
  "ruwordnet_vectors_path": "ruwordnet_verbs.txt",
  "data_vectors_path": "verbs_private.txt",
  "test_path": "taxonomy_enrichment/data/private_test/verbs_private.tsv",
  "output_path": "predicted_private_verbs_fasttext.tsv",
  "db_path": "ruwordnet.db",
  "ruwordnet_path": None
}

from taxonomy_enrichment.baselines.main import *

with open(params['test_path'], 'r', encoding='utf-8') as f:
    test_data = f.read().split("\n")[:-1]
  
baseline = SecondOrderModel(params)
results = baseline.predict_hypernyms(list(test_data))
save_to_file(results, params['output_path'], baseline.ruwordnet)

## ELMo

In [21]:
from ruwordnet.ruwordnet_reader import RuWordnet

ruwordnet = RuWordnet(db_path='ruwordnet.db', ruwordnet_path="")

In [22]:
!pip install simple_elmo


     |████████████████████████████████| 46 kB 3.0 MB/s 


In [23]:
!wget -O tayga_lemmas_elmo_2048_2019.zip http://vectors.nlpl.eu/repository/20/199.zip
!unzip tayga_lemmas_elmo_2048_2019.zip -d tayga_lemmas_elmo_2048_2019

--2021-12-22 00:17:14--  http://vectors.nlpl.eu/repository/20/199.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1800442066 (1.7G) [application/zip]
Saving to: ‘tayga_lemmas_elmo_2048_2019.zip’

tayga_lemmas_elmo_2 100%[===================>]   1.68G  64.4MB/s    in 1m 59s  

2021-12-22 00:19:13 (14.4 MB/s) - ‘tayga_lemmas_elmo_2048_2019.zip’ saved [1800442066/1800442066]

Archive:  tayga_lemmas_elmo_2048_2019.zip
  inflating: tayga_lemmas_elmo_2048_2019/meta.json  
  inflating: tayga_lemmas_elmo_2048_2019/model.ckpt.data-00000-of-00001  
  inflating: tayga_lemmas_elmo_2048_2019/model.ckpt.index  
  inflating: tayga_lemmas_elmo_2048_2019/model.ckpt.meta  
  inflating: tayga_lemmas_elmo_2048_2019/model.hdf5  
  inflating: tayga_lemmas_elmo_2048_2019/options.json  
  inflating: tayga_lemmas_elmo_2048_2019/README  
  inflating: tayga_l

In [24]:
import numpy as np
from tqdm import tqdm
from collections import defaultdict

def save_as_w2v(vectors_dict, output_path):
    with open(output_path, 'w', encoding='utf-8') as w:
        w.write(f"{len(vectors_dict)} {len( vectors_dict[list(vectors_dict.keys())[0]] )}\n")
        for key in vectors_dict:
            vector_line = " ".join(map(str, vectors_dict[key]))
            w.write(f"{key.upper()} {vector_line}\n")

def elmo_vectorize(model, data, output_path):
    batch_size = model.batch_size
    result = defaultdict(list)

    keys_b = []
    value_b = []
    for k in tqdm(data, position=0, leave=True):
        keys_b.append(k)
        value_b.append(data[k])
        if len(keys_b) != batch_size:
            continue

        res = model.get_elmo_vector_average(value_b);
        for i, ib in enumerate(keys_b):
            result[ib] = res[i]
        keys_b = []
        value_b = []
    
    if keys_b:
        res = model.get_elmo_vector_average(value_b);
        for i, ib in enumerate(keys_b):
            result[ib] = res[i]
    
    save_as_w2v(result, output_path)

def process_data(model, input_file, output_file):
    data = defaultdict(list)
    with open(input_file, 'r', encoding='utf-8') as f:
        dataset = f.read().lower().split("\n")[:-1]
    for i in dataset:
        data[i] = i
    elmo_vectorize(model, data, output_file)

In [25]:
from simple_elmo import ElmoModel

model = ElmoModel()
model.load('tayga_lemmas_elmo_2048_2019', max_batch_size=100)

2021-12-22 00:28:41,064 : INFO : Loading model from tayga_lemmas_elmo_2048_2019...
2021-12-22 00:28:41,068 : INFO : We will cache the vocabulary of 100 tokens.
/usr/local/lib/python3.7/dist-packages/simple_elmo/model.py:531: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell_clip=cell_clip, proj_clip=proj_clip)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)
/usr/local/lib/python3.7/dist-pac

'The model is now loaded.'

In [26]:
import logging
model.logger.setLevel(logging.CRITICAL)

In [27]:
from collections import defaultdict

noun_synsets = defaultdict(list)
verb_synsets = defaultdict(list)
for sense_id, synset_id, text in ruwordnet.get_all_senses():
    if synset_id.endswith("N"):
        noun_synsets[synset_id].append(text.lower())
    elif synset_id.endswith("V"):
        verb_synsets[synset_id].append(text.lower())

In [28]:
elmo_vectorize(model, noun_synsets, 'ruwordnet_nouns_elmo.txt')
elmo_vectorize(model, verb_synsets, 'ruwordnet_verbs_elmo.txt')

100%|██████████| 7521/7521 [05:17<00:00, 23.69it/s]


In [29]:
process_data(model, "taxonomy_enrichment/data/public_test/verbs_public.tsv", "verbs_public_elmo.txt")
process_data(model, "taxonomy_enrichment/data/public_test/nouns_public.tsv", "nouns_public_elmo.txt")
process_data(model, "taxonomy_enrichment/data/private_test/verbs_private.tsv", "verbs_private_elmo.txt")
process_data(model, "taxonomy_enrichment/data/private_test/nouns_private.tsv", "nouns_private_elmo.txt")

100%|██████████| 1525/1525 [01:00<00:00, 25.28it/s]


In [30]:
params = {
  "ruwordnet_vectors_path": "ruwordnet_nouns_elmo.txt",
  "data_vectors_path": "nouns_public_elmo.txt",
  "test_path": "taxonomy_enrichment/data/public_test/nouns_public.tsv",
  "output_path": "predicted_public_nouns_elmo.tsv",
  "db_path": "ruwordnet.db",
  "ruwordnet_path": None
}

from taxonomy_enrichment.baselines.main import *

with open(params['test_path'], 'r', encoding='utf-8') as f:
    test_data = f.read().split("\n")[:-1]
  
baseline = SecondOrderModel(params)
results = baseline.predict_hypernyms(list(test_data))
save_to_file(results, params['output_path'], baseline.ruwordnet)

2021-12-22 01:03:12,081 : INFO : loading projection weights from ruwordnet_nouns_elmo.txt
2021-12-22 01:04:03,395 : INFO : loaded (29296, 1024) matrix from ruwordnet_nouns_elmo.txt
2021-12-22 01:04:03,402 : INFO : loading projection weights from nouns_public_elmo.txt
2021-12-22 01:04:04,659 : INFO : loaded (762, 1024) matrix from nouns_public_elmo.txt
2021-12-22 01:04:04,661 : INFO : precomputing L2-norms of word weight vectors


In [31]:
params = {
  "ruwordnet_vectors_path": "ruwordnet_verbs_elmo.txt",
  "data_vectors_path": "verbs_public_elmo.txt",
  "test_path": "taxonomy_enrichment/data/public_test/verbs_public.tsv",
  "output_path": "predicted_public_verbs_elmo.tsv",
  "db_path": "ruwordnet.db",
  "ruwordnet_path": None
}

from taxonomy_enrichment.baselines.main import *

with open(params['test_path'], 'r', encoding='utf-8') as f:
    test_data = f.read().split("\n")[:-1]
  
baseline = SecondOrderModel(params)
results = baseline.predict_hypernyms(list(test_data))
save_to_file(results, params['output_path'], baseline.ruwordnet)

2021-12-22 01:04:37,968 : INFO : loading projection weights from ruwordnet_verbs_elmo.txt
2021-12-22 01:04:49,736 : INFO : loaded (7521, 1024) matrix from ruwordnet_verbs_elmo.txt
2021-12-22 01:04:49,739 : INFO : loading projection weights from verbs_public_elmo.txt
2021-12-22 01:04:50,022 : INFO : loaded (175, 1024) matrix from verbs_public_elmo.txt
2021-12-22 01:04:50,026 : INFO : precomputing L2-norms of word weight vectors


In [32]:
params = {
  "ruwordnet_vectors_path": "ruwordnet_nouns_elmo.txt",
  "data_vectors_path": "nouns_private_elmo.txt",
  "test_path": "taxonomy_enrichment/data/private_test/nouns_private.tsv",
  "output_path": "predicted_private_nouns_elmo.tsv",
  "db_path": "ruwordnet.db",
  "ruwordnet_path": None
}

from taxonomy_enrichment.baselines.main import *

with open(params['test_path'], 'r', encoding='utf-8') as f:
    test_data = f.read().split("\n")[:-1]
  
baseline = SecondOrderModel(params)
results = baseline.predict_hypernyms(list(test_data))
save_to_file(results, params['output_path'], baseline.ruwordnet)

2021-12-22 01:04:50,639 : INFO : loading projection weights from ruwordnet_nouns_elmo.txt
2021-12-22 01:05:39,821 : INFO : loaded (29296, 1024) matrix from ruwordnet_nouns_elmo.txt
2021-12-22 01:05:39,825 : INFO : loading projection weights from nouns_private_elmo.txt
2021-12-22 01:05:44,349 : INFO : loaded (1525, 1024) matrix from nouns_private_elmo.txt
2021-12-22 01:05:44,356 : INFO : precomputing L2-norms of word weight vectors


In [33]:
params = {
  "ruwordnet_vectors_path": "ruwordnet_verbs_elmo.txt",
  "data_vectors_path": "verbs_private_elmo.txt",
  "test_path": "taxonomy_enrichment/data/private_test/verbs_private.tsv",
  "output_path": "predicted_private_verbs_elmo.tsv",
  "db_path": "ruwordnet.db",
  "ruwordnet_path": None
}

from taxonomy_enrichment.baselines.main import *

with open(params['test_path'], 'r', encoding='utf-8') as f:
    test_data = f.read().split("\n")[:-1]
  
baseline = SecondOrderModel(params)
results = baseline.predict_hypernyms(list(test_data))
save_to_file(results, params['output_path'], baseline.ruwordnet)

2021-12-22 01:06:11,028 : INFO : loading projection weights from ruwordnet_verbs_elmo.txt
2021-12-22 01:06:21,752 : INFO : loaded (7521, 1024) matrix from ruwordnet_verbs_elmo.txt
2021-12-22 01:06:21,754 : INFO : loading projection weights from verbs_private_elmo.txt
2021-12-22 01:06:22,323 : INFO : loaded (350, 1024) matrix from verbs_private_elmo.txt
2021-12-22 01:06:22,333 : INFO : precomputing L2-norms of word weight vectors
